# Converting Pointclouds to STLs using Open3D

Make sure to have markupsafe version 2.0.1, open3d will attempt to install a newer version when installed.

In [1]:
import open3d as o3d
import trimesh
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Here a box is created using Trimesh for testing puposes.

(If the model doesn't show restart the kernel until it does)

In [2]:
cube_mesh = trimesh.creation.box((1,1,1))
cube_mesh.show()

Here a pointcloud is created out of the cube mesh as an example point cloud

In [3]:
num_points = 50000
# Samples a buch of random points on the surface of the cube
points, face_indices = cube_mesh.sample(num_points, return_index=True)

# Creates the pointcloud
point_cloud = trimesh.PointCloud(vertices=points)
point_cloud.export('models/cube.ply')

b'ply\nformat binary_little_endian 1.0\ncomment https://github.com/mikedh/trimesh\nelement vertex 50000\nproperty float x\nproperty float y\nproperty float z\nproperty uchar red\nproperty uchar green\nproperty uchar blue\nproperty uchar alpha\nend_header\n)5(\xbc\xc6\xeak\xbe\x00\x00\x00\xbf\x00\x00\x00\x00}\xaa\xfc>\x00\x00\x00?|D\xf1\xbe\x00\x00\x00\x00\xe5\xdap=w\x12q\xbe\x00\x00\x00\xbf\x00\x00\x00\x00\x00\x00\x00?\xa1Zp>sZ\xe0\xbd\x00\x00\x00\x00#\xbb\x81\xbe\x00\x00\x00?\x96\x86\x13\xbe\x00\x00\x00\x00n\x9bf\xbe\x00\x00\x00\xbf4\xf4\x9a>\x00\x00\x00\x00\x00\x00\x00\xbf\xd9.p\xbe^\xf3\xac=\x00\x00\x00\x00\x93\x81p>\xd3l\x8d>\x00\x00\x00\xbf\x00\x00\x00\x00\xf9\xfc\\\xbe\x00\x00\x00\xbf$\x87\xb7=\x00\x00\x00\x00\x00\x00\x00?\xe9\x14\xaa\xbe\xb5\t$\xbe\x00\x00\x00\x00\x10\xdf#>\x00\x00\x00?\xdd\x98\x8c>\x00\x00\x00\x00\xd0\xedo>\xe9\x9a\xd2\xbe\x00\x00\x00?\x00\x00\x00\x00\x00\x00\x00?\xc2\x13\xbb\xbeM\x0b\xd3\xbd\x00\x00\x00\x00\xa0\xda\x94>\x00\x00\x00?\x1bN\xe0>\x00\x00\x00\x00N\

The below functions takes in the path to a pointcloud file and converts it to a STL.

In [4]:
def pointcloud_to_stl(file_name, output_name="output_model.stl"):
    """
        Converts a point cloud file to an STL.

        @param file_path (str): Path to the point cloud file.
        @param output_name (str):  Name for the output STL file.
        **reconstruction_kwargs: Keyword arguments for the reconstruction algorithm
                                 (e.g., depth, voxel_size, etc.)
    """

    # Load point cloud with Trimesh
    point_cloud = trimesh.load(file_name)
    vertices = np.asarray(point_cloud.vertices)
    center = np.mean(vertices, axis=0)
    infinity_point = center * 1000
    vertices = np.vstack([vertices, infinity_point])

    # Surface reconstruction using Open3D
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(vertices)
    pcd.estimate_normals()
    pcd.orient_normals_consistent_tangent_plane(k=15)

    with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Error) as cm:
        mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=11)

    vertices = np.asarray(mesh.vertices)
    faces = np.asarray(mesh.triangles)

    # Create Trimesh object
    output_mesh = trimesh.Trimesh(vertices=vertices, faces=faces)

    # Export as STL
    output_mesh.export(output_name)

In [5]:
pointcloud_to_stl('models/cube.ply', output_name='models/ply_model.stl')

Here the pointcloud is loaded and smoothed by trimesh to reduce some of the noise.

In [6]:
mesh = trimesh.load('models/ply_model.stl')
mesh_smoothed = trimesh.smoothing.filter_laplacian(mesh, iterations=5)

In [7]:
mesh.show()

In [ ]:
mesh_smoothed.show()